In [1]:
import clingo
def print_answer_sets(program):
    # Load the answer set program, and call the grounder
    control = clingo.Control()
    control.add("base", [], program)
    control.ground([("base", [])])
    # Define a function that will be called when an answer set is found
    # This function sorts the answer set alphabetically, and prints it
    def on_model(model):
        sorted_model = [str(atom) for atom in model.symbols(shown=True)]
        sorted_model.sort()
        print("Answer set: {{{}}}".format(", ".join(sorted_model)))
    # Ask clingo to find all models (using an upper bound of 0 gives all models)
    control.configuration.solve.models = 0
    # Call the clingo solver, passing on the function on_model for when an answer set is found
    answer = control.solve(on_model=on_model)
    # Print a message when no answer set was found
    if answer.satisfiable == False:
        print("No answer sets")
def print_optimal_answer_sets(program):
    # Load the answer set program, and call the grounder
    control = clingo.Control()
    control.add("base", [], program)
    control.ground([("base", [])])
    # Define a function that will be called when an answer set is found
    # This function sorts the answer set alphabetically, and prints it
    def on_model(model):
        if model.optimality_proven == True:
            sorted_model = [str(atom) for atom in model.symbols(shown=True)]
            sorted_model.sort()
            print("Optimal answer set: {{{}}}".format(", ".join(sorted_model)))
    # Ask clingo to find all optimal models (using an upper bound of 0 gives all models)
    control.configuration.solve.opt_mode = "optN"
    control.configuration.solve.models = 0
    # Call the clingo solver, passing on the function on_model for when an answer set is found
    answer = control.solve(on_model=on_model)
    # Print a message when no answer set was found
    if answer.satisfiable == False:
        print("No answer sets")

In [2]:
def print_optimal_answer_sets(program):
    # Load the answer set program, and call the grounder
    control = clingo.Control()
    control.add("base", [], program)
    control.ground([("base", [])])
    # Define a function that will be called when an answer set is found
    # This function sorts the answer set alphabetically, and prints it
    def on_model(model):
        if model.optimality_proven == True:
            sorted_model = [str(atom) for atom in model.symbols(shown=True)]
            sorted_model.sort()
            print("Optimal answer set: {{{}}}".format(", ".join(sorted_model)))
    # Ask clingo to find all optimal models (using an upper bound of 0 gives all models)
    control.configuration.solve.opt_mode = "optN"
    control.configuration.solve.models = 0
    # Call the clingo solver, passing on the function on_model for when an answer set is found
    answer = control.solve(on_model=on_model)
    # Print a message when no answer set was found
    if answer.satisfiable == False:
        print("No answer sets")

In [3]:
print_answer_sets("""
% INITIALIZE GRAPH COORDINATES
node(1..8).
coord(1,0,0).
coord(2,3,1).
coord(3,1,3).
coord(4,2,5).
coord(5,4,8).
coord(6,6,6).
coord(7,10,6).
coord(8,5,3).
dist(N1,N2,X) :- node(N1), node(N2), N1!=N2, coord(N1,AX,AY), coord(N2,BX,BY), X=(BX-AX)**2+(BY-AY)**2.
sdist(N1,N2,X) :- dist(N1,N2,X), node(N1), node(N2), N2>N1.

% INITIALIZE EVENTS: event(ID, NODE_NR)
#const num_events=2.
event(1..num_events).
event(1,7).
event(2,1).

% INITIALIZE PARTICIPANTS: participant(ID, NODE_NR, EVENT_NR)
#const num_participants=4.
participant(1..num_participants).
participant(1,5,2).
participant(2,4,1).
participant(3,8,1).
participant(4,2,2).

% INITIALIZE DRIVERS: DRIVER(ID, NODE_NR)
#const num_drivers=2.
driver(1..num_drivers).
driver(1,6).
driver(2,3).

% GENERATE PICKUP(driverID, T, participantID)
#const num_timesteps=4.
timestep(1..4).
%1{ pickup(1..D, 1..T, 1..P) : driver(D), timestep(T), participant(P) }num_participants .%: driver(D).%, timestep(T).
{pickup(D, T, P) : driver(D), timestep(T), participant(P)}.

% DIFFERENT DRIVERS CANNOT PICKUP THE SAME PARTICIPANT
:- pickup(D1,_,X), pickup(D2,_,X), driver(D1), driver(D2), participant(X), D1!=D2.

% SAME DRIVER CANNOT PICKUP SAME PARTICIPANT TWICE
:- pickup(D,T1,X), pickup(D,T2,X), driver(D), timestep(T1), timestep(T2), participant(X), T1!=T2.

% SAME DRIVER CANNOT HAVE SAME TIMESTAMPS
:- pickup(D,T1,X1), pickup(D,T2,X2), driver(D), timestep(T1), timestep(T2), participant(X1), participant(X2), T1==T2, X1!=X2.

% EACH DRIVER STARTS AT TIMESTEP 1
:- not pickup(D,1,_), pickup(D,T,X).

% PICKUP TIMES MUST BE CONSECUTIVE FOR EACH DRIVER
:- pickup(D,T1,_), pickup(D,T2,_), T2-T1>1, not pickup(D,T1+1,_).

% ALL PARTICIPANTS MUST BE PICKED UP
picked_up(N) :- N = #count{ ID : pickup(_,_,ID) }.
:- picked_up(N), N<4.

% EACH DRIVER CAN ONLY PICK UP A MAXIUM OF 3
picked_up(D,N) :- N = #count{ ID : pickup(D,_,ID)}, driver(D).
:- picked_up(D,N), driver(D), N>3.

% EACH PASSENGER MUST HAVE THE SAME DESTINATION
targets(D,E) :- pickup(D,_,P), participant(P,_,E).
:- N=#count{TGT : targets(D,TGT)}, driver(D), N>1.

% ACCOUNT FOR TRAVEL DISTANCES PER DRIVER
distances(D,0,KM) :- driver(D), pickup(D,1,P), driver(D,SRC), participant(P,TGT,E), dist(SRC,TGT,KM).
distances(D,T1,KM) :- driver(D), pickup(D,T1,P1), pickup(D,T2,P2), T2-T1=1, participant(P1,SRC,E), participant(P2,TGT,E), dist(SRC,TGT,KM).
max_t(D,T) :- T = #max{ TT : pickup(D,TT,_) }, driver(D).
distances(D,TMAX,KM) :- max_t(D,TMAX), driver(D), pickup(D,TMAX,P), participant(P,SRC,EVT), event(EVT,TGT), dist(SRC,TGT,KM).

%distances_per_driver(D,KM) :- KM = #sum{ K : distances(D,_,K)}, driver(D).
distances(KM) :- KM = #sum{ K,D : distances(D,_,K), driver(D)}.
%total_distance(KM) :- KM = #sum{ K,D : distances_per_driver(D,K)}, driver(D).

%#show max_t/2.
%#show distances/3.
#show distances/1.
%#show targets/2.
%#show picked_up/2.
#show pickup/3.
%#show picked_up/1.
%#show dist/3.
%#show node/1.
%#show sdist/3.
%#show driver/1.
%#show driver/2.
""")

Answer set: {distances(182), pickup(1,1,3), pickup(1,2,2), pickup(2,1,1), pickup(2,2,4)}
Answer set: {distances(226), pickup(1,1,3), pickup(1,2,2), pickup(2,1,4), pickup(2,2,1)}
Answer set: {distances(258), pickup(1,1,4), pickup(1,2,1), pickup(2,1,3), pickup(2,2,2)}
Answer set: {distances(216), pickup(1,1,4), pickup(1,2,1), pickup(2,1,2), pickup(2,2,3)}
Answer set: {distances(202), pickup(1,1,2), pickup(1,2,3), pickup(2,1,4), pickup(2,2,1)}
Answer set: {distances(158), pickup(1,1,2), pickup(1,2,3), pickup(2,1,1), pickup(2,2,4)}
Answer set: {distances(162), pickup(1,1,1), pickup(1,2,4), pickup(2,1,3), pickup(2,2,2)}
Answer set: {distances(120), pickup(1,1,1), pickup(1,2,4), pickup(2,1,2), pickup(2,2,3)}


In [19]:
print_optimal_answer_sets("""
% INITIALIZE GRAPH COORDINATES
node(1..8).
coord(1,0,0).
coord(2,3,1).
coord(3,1,3).
coord(4,2,5).
coord(5,4,8).
coord(6,6,6).
coord(7,10,6).
coord(8,5,3).
dist(N1,N2,X) :- node(N1), node(N2), N1!=N2, coord(N1,AX,AY), coord(N2,BX,BY), X=(BX-AX)**2+(BY-AY)**2.
sdist(N1,N2,X) :- dist(N1,N2,X), node(N1), node(N2), N2>N1.

% INITIALIZE EVENTS: event(ID, NODE_NR)
#const num_events=2.
event(1..num_events).
event(1,7).
event(2,1).

% INITIALIZE PARTICIPANTS: participant(ID, NODE_NR, EVENT_NR)
#const num_participants=4.
participant(1..num_participants).
participant(1,5,2).
participant(2,4,1).
participant(3,8,1).
participant(4,2,2).

% INITIALIZE DRIVERS: DRIVER(ID, NODE_NR)
#const num_drivers=2.
driver(1..num_drivers).
driver(1,6).
driver(2,3).

% GENERATE PICKUP(driverID, T, participantID)
#const num_timesteps=4.
timestep(1..4).
%1{ pickup(1..D, 1..T, 1..P) : driver(D), timestep(T), participant(P) }num_participants .%: driver(D).%, timestep(T).
{pickup(D, T, P) : driver(D), timestep(T), participant(P)}.

% DIFFERENT DRIVERS CANNOT PICKUP THE SAME PARTICIPANT
:- pickup(D1,_,X), pickup(D2,_,X), driver(D1), driver(D2), participant(X), D1!=D2.

% SAME DRIVER CANNOT PICKUP SAME PARTICIPANT TWICE
:- pickup(D,T1,X), pickup(D,T2,X), driver(D), timestep(T1), timestep(T2), participant(X), T1!=T2.

% SAME DRIVER CANNOT HAVE SAME TIMESTAMPS
:- pickup(D,T1,X1), pickup(D,T2,X2), driver(D), timestep(T1), timestep(T2), participant(X1), participant(X2), T1==T2, X1!=X2.

% EACH DRIVER STARTS AT TIMESTEP 1
:- not pickup(D,1,_), pickup(D,T,X).

% PICKUP TIMES MUST BE CONSECUTIVE FOR EACH DRIVER
:- pickup(D,T1,_), pickup(D,T2,_), T2-T1>1, not pickup(D,T1+1,_).

% ALL PARTICIPANTS MUST BE PICKED UP
picked_up(N) :- N = #count{ ID : pickup(_,_,ID) }.
:- picked_up(N), N<4.

% EACH DRIVER CAN ONLY PICK UP A MAXIUM OF 3
picked_up(D,N) :- N = #count{ ID : pickup(D,_,ID)}, driver(D).
:- picked_up(D,N), driver(D), N>3.

% EACH PASSENGER MUST HAVE THE SAME DESTINATION
targets(D,E) :- pickup(D,_,P), participant(P,_,E).
:- N=#count{TGT : targets(D,TGT)}, driver(D), N>1.

% ACCOUNT FOR TRAVEL DISTANCES PER DRIVER
distances(D,0,KM) :- driver(D), pickup(D,1,P), driver(D,SRC), participant(P,TGT,E), dist(SRC,TGT,KM).
distances(D,T1,KM) :- driver(D), pickup(D,T1,P1), pickup(D,T2,P2), T2-T1=1, participant(P1,SRC,E), participant(P2,TGT,E), dist(SRC,TGT,KM).
max_t(D,T) :- T = #max{ TT : pickup(D,TT,_) }, driver(D).
distances(D,TMAX,KM) :- max_t(D,TMAX), driver(D), pickup(D,TMAX,P), participant(P,SRC,EVT), event(EVT,TGT), dist(SRC,TGT,KM).

%distances_per_driver(D,KM) :- KM = #sum{ K : distances(D,_,K)}, driver(D).
distances(KM) :- KM = #sum{ K,D : distances(D,_,K), driver(D)}.
%total_distance(KM) :- KM = #sum{ K,D : distances_per_driver(D,K)}, driver(D).

#minimize {KM : distances(KM)}.

%#show max_t/2.
%#show distances/3.
#show distances/1.
%#show targets/2.
%#show picked_up/2.
#show pickup/3.
%#show picked_up/1.
%#show dist/3.
%#show node/1.
%#show sdist/3.
%#show driver/1.
%#show driver/2.
""")

Optimal answer set: {distances(120), pickup(1,1,1), pickup(1,2,4), pickup(2,1,2), pickup(2,2,3)}


In [21]:
print_answer_sets("""
% INITIALIZE GRAPH COORDINATES
node(1..8).
coord(1,0,0).
coord(2,4,3).
coord(3,4,2).
coord(4,6,5).
coord(5,6,1).
coord(6,7,5).
coord(7,8,3).
coord(8,9,5).
dist(N1,N2,X) :- node(N1), node(N2), N1!=N2, coord(N1,AX,AY), coord(N2,BX,BY), X=(BX-AX)**2+(BY-AY)**2.
sdist(N1,N2,X) :- dist(N1,N2,X), node(N1), node(N2), N2>N1.

% INITIALIZE EVENTS: event(ID, NODE_NR)
#const num_events=2.
event(1..num_events).
event(1,1).
event(2,8).

% INITIALIZE PARTICIPANTS: participant(ID, NODE_NR, EVENT_NR)
#const num_participants=4.
participant(1..num_participants).
participant(1,4,2).
participant(2,6,2).
participant(3,7,2).
participant(4,5,1).

% INITIALIZE DRIVERS: DRIVER(ID, NODE_NR)
#const num_drivers=2.
driver(1..num_drivers).
driver(1,2).
driver(2,3).

% GENERATE PICKUP(driverID, T, participantID)
#const num_timesteps=4.
timestep(1..4).
%1{ pickup(1..D, 1..T, 1..P) : driver(D), timestep(T), participant(P) }num_participants .%: driver(D).%, timestep(T).
{pickup(D, T, P) : driver(D), timestep(T), participant(P)}.

% DIFFERENT DRIVERS CANNOT PICKUP THE SAME PARTICIPANT
:- pickup(D1,_,X), pickup(D2,_,X), driver(D1), driver(D2), participant(X), D1!=D2.

% SAME DRIVER CANNOT PICKUP SAME PARTICIPANT TWICE
:- pickup(D,T1,X), pickup(D,T2,X), driver(D), timestep(T1), timestep(T2), participant(X), T1!=T2.

% SAME DRIVER CANNOT HAVE SAME TIMESTAMPS
:- pickup(D,T1,X1), pickup(D,T2,X2), driver(D), timestep(T1), timestep(T2), participant(X1), participant(X2), T1==T2, X1!=X2.

% EACH DRIVER STARTS AT TIMESTEP 1
:- not pickup(D,1,_), pickup(D,T,X).

% PICKUP TIMES MUST BE CONSECUTIVE FOR EACH DRIVER
:- pickup(D,T1,_), pickup(D,T2,_), T2-T1>1, not pickup(D,T1+1,_).

% ALL PARTICIPANTS MUST BE PICKED UP
picked_up(N) :- N = #count{ ID : pickup(_,_,ID) }.
:- picked_up(N), N<4.

% EACH DRIVER CAN ONLY PICK UP A MAXIUM OF 3
picked_up(D,N) :- N = #count{ ID : pickup(D,_,ID)}, driver(D).
:- picked_up(D,N), driver(D), N>3.

% EACH PASSENGER MUST HAVE THE SAME DESTINATION
targets(D,E) :- pickup(D,_,P), participant(P,_,E).
:- N=#count{TGT : targets(D,TGT)}, driver(D), N>1.

% ACCOUNT FOR TRAVEL DISTANCES PER DRIVER
distances(D,0,KM) :- driver(D), pickup(D,1,P), driver(D,SRC), participant(P,TGT,E), dist(SRC,TGT,KM).
distances(D,T1,KM) :- driver(D), pickup(D,T1,P1), pickup(D,T2,P2), T2-T1=1, participant(P1,SRC,E), participant(P2,TGT,E), dist(SRC,TGT,KM).
max_t(D,T) :- T = #max{ TT : pickup(D,TT,_) }, driver(D).
distances(D,TMAX,KM) :- max_t(D,TMAX), driver(D), pickup(D,TMAX,P), participant(P,SRC,EVT), event(EVT,TGT), dist(SRC,TGT,KM).

%distances_per_driver(D,KM) :- KM = #sum{ K : distances(D,_,K)}, driver(D).
distances(KM) :- KM = #sum{ K,D,T : distances(D,T,K), driver(D)}.
%total_distance(KM) :- KM = #sum{ K,D : distances_per_driver(D,K)}, driver(D).

%#show max_t/2.
%#show distances/3.
#show distances/1.
%#show targets/2.
%#show picked_up/2.
#show pickup/3.
%#show picked_up/1.
%#show dist/3.
%#show node/1.
%#show sdist/3.
%#show driver/1.
%#show driver/2.
""")

Answer set: {distances(67), pickup(1,1,1), pickup(1,2,3), pickup(1,3,2), pickup(2,1,4)}
Answer set: {distances(71), pickup(1,1,3), pickup(1,2,1), pickup(1,3,2), pickup(2,1,4)}
Answer set: {distances(61), pickup(1,1,1), pickup(1,2,2), pickup(1,3,3), pickup(2,1,4)}
Answer set: {distances(69), pickup(1,1,2), pickup(1,2,1), pickup(1,3,3), pickup(2,1,4)}
Answer set: {distances(77), pickup(1,1,2), pickup(1,2,3), pickup(1,3,1), pickup(2,1,4)}
Answer set: {distances(73), pickup(1,1,3), pickup(1,2,2), pickup(1,3,1), pickup(2,1,4)}
Answer set: {distances(69), pickup(1,1,4), pickup(2,1,1), pickup(2,2,2), pickup(2,3,3)}
Answer set: {distances(75), pickup(1,1,4), pickup(2,1,1), pickup(2,2,3), pickup(2,3,2)}
Answer set: {distances(77), pickup(1,1,4), pickup(2,1,2), pickup(2,2,1), pickup(2,3,3)}
Answer set: {distances(85), pickup(1,1,4), pickup(2,1,2), pickup(2,2,3), pickup(2,3,1)}
Answer set: {distances(77), pickup(1,1,4), pickup(2,1,3), pickup(2,2,2), pickup(2,3,1)}
Answer set: {distances(75), pick

In [22]:
print_optimal_answer_sets("""
% INITIALIZE GRAPH COORDINATES
node(1..8).
coord(1,0,0).
coord(2,4,3).
coord(3,4,2).
coord(4,6,5).
coord(5,6,1).
coord(6,7,5).
coord(7,8,3).
coord(8,9,5).
coord(9,8,2).
dist(N1,N2,X) :- node(N1), node(N2), N1!=N2, coord(N1,AX,AY), coord(N2,BX,BY), X=(BX-AX)**2+(BY-AY)**2.
sdist(N1,N2,X) :- dist(N1,N2,X), node(N1), node(N2), N2>N1.

% INITIALIZE EVENTS: event(ID, NODE_NR)
#const num_events=2.
event(1..num_events).
event(1,1).
event(2,8).

% INITIALIZE PARTICIPANTS: participant(ID, NODE_NR, EVENT_NR)
#const num_participants=4.
participant(1..num_participants).
participant(1,4,2).
participant(2,6,2).
participant(3,7,2).
participant(4,5,1).

% INITIALIZE DRIVERS: DRIVER(ID, NODE_NR)
#const num_drivers=3.
driver(1..num_drivers).
driver(1,2).
driver(2,3).
driver(3,7).

% GENERATE PICKUP(driverID, T, participantID)
#const num_timesteps=4.
timestep(1..4).
%1{ pickup(1..D, 1..T, 1..P) : driver(D), timestep(T), participant(P) }num_participants .%: driver(D).%, timestep(T).
{pickup(D, T, P) : driver(D), timestep(T), participant(P)}.

% DIFFERENT DRIVERS CANNOT PICKUP THE SAME PARTICIPANT
:- pickup(D1,_,X), pickup(D2,_,X), driver(D1), driver(D2), participant(X), D1!=D2.

% SAME DRIVER CANNOT PICKUP SAME PARTICIPANT TWICE
:- pickup(D,T1,X), pickup(D,T2,X), driver(D), timestep(T1), timestep(T2), participant(X), T1!=T2.

% SAME DRIVER CANNOT HAVE SAME TIMESTAMPS
:- pickup(D,T1,X1), pickup(D,T2,X2), driver(D), timestep(T1), timestep(T2), participant(X1), participant(X2), T1==T2, X1!=X2.

% EACH DRIVER STARTS AT TIMESTEP 1
:- not pickup(D,1,_), pickup(D,T,X).

% PICKUP TIMES MUST BE CONSECUTIVE FOR EACH DRIVER
:- pickup(D,T1,_), pickup(D,T2,_), T2-T1>1, not pickup(D,T1+1,_).

% ALL PARTICIPANTS MUST BE PICKED UP
picked_up(N) :- N = #count{ ID : pickup(_,_,ID) }.
:- picked_up(N), N<4.

% EACH DRIVER CAN ONLY PICK UP A MAXIUM OF 3
picked_up(D,N) :- N = #count{ ID : pickup(D,_,ID)}, driver(D).
:- picked_up(D,N), driver(D), N>3.

% EACH PASSENGER MUST HAVE THE SAME DESTINATION
targets(D,E) :- pickup(D,_,P), participant(P,_,E).
:- N=#count{TGT : targets(D,TGT)}, driver(D), N>1.

% ACCOUNT FOR TRAVEL DISTANCES PER DRIVER
distances(D,0,KM) :- driver(D), pickup(D,1,P), driver(D,SRC), participant(P,TGT,E), dist(SRC,TGT,KM).
distances(D,T1,KM) :- driver(D), pickup(D,T1,P1), pickup(D,T2,P2), T2-T1=1, participant(P1,SRC,E), participant(P2,TGT,E), dist(SRC,TGT,KM).
max_t(D,T) :- T = #max{ TT : pickup(D,TT,_) }, driver(D).
distances(D,TMAX,KM) :- max_t(D,TMAX), driver(D), pickup(D,TMAX,P), participant(P,SRC,EVT), event(EVT,TGT), dist(SRC,TGT,KM).

%distances_per_driver(D,KM) :- KM = #sum{ K : distances(D,_,K)}, driver(D).
distances(KM) :- KM = #sum{ K,D,T : distances(D,T,K), driver(D)}.
%total_distance(KM) :- KM = #sum{ K,D : distances_per_driver(D,K)}, driver(D).

#minimize {KM : distances(KM)}.

%#show max_t/2.
%#show distances/3.
#show distances/1.
%#show targets/2.
%#show picked_up/2.
#show pickup/3.
%#show picked_up/1.
#show dist/3.
%#show node/1.
%#show sdist/3.
%#show driver/1.
%#show driver/2.
""")

Optimal answer set: {dist(1,2,25), dist(1,3,20), dist(1,4,61), dist(1,5,37), dist(1,6,74), dist(1,7,73), dist(1,8,106), dist(2,1,25), dist(2,3,1), dist(2,4,8), dist(2,5,8), dist(2,6,13), dist(2,7,16), dist(2,8,29), dist(3,1,20), dist(3,2,1), dist(3,4,13), dist(3,5,5), dist(3,6,18), dist(3,7,17), dist(3,8,34), dist(4,1,61), dist(4,2,8), dist(4,3,13), dist(4,5,16), dist(4,6,1), dist(4,7,8), dist(4,8,9), dist(5,1,37), dist(5,2,8), dist(5,3,5), dist(5,4,16), dist(5,6,17), dist(5,7,8), dist(5,8,25), dist(6,1,74), dist(6,2,13), dist(6,3,18), dist(6,4,1), dist(6,5,17), dist(6,7,5), dist(6,8,4), dist(7,1,73), dist(7,2,16), dist(7,3,17), dist(7,4,8), dist(7,5,8), dist(7,6,5), dist(7,8,5), dist(8,1,106), dist(8,2,29), dist(8,3,34), dist(8,4,9), dist(8,5,25), dist(8,6,4), dist(8,7,5), distances(55), pickup(2,1,4), pickup(3,1,3), pickup(3,2,1), pickup(3,3,2)}
